<img src="../idp.jpg">

# <p style="background-color:#2F5597;font-family:newtimeroman;color:#FFF9ED;font-size:150%;text-align:center;border-radius:10px 10px;">Aula 5 - Risco de fraude de crédito</p>

<h2> Introdução </h2>
Neste projeto vamos aplicar algoritmos de inteligência artificial para a detecção de fraudes em cartões de crédito, um problema muito grande que as instituições financeiras e Fintechs têm enfrentado diariamente, que é identificar se uma transação é uma fraude ou não. É de extrema importância que as operadoras de cartão de crédito estejam preparadas para esse tipo de crime, monitorando constantemente o comportamento dos cartões.

<h2> Objetivos desse projeto </h2>
O objetivo é fazer uso do aprendizado de máquina para identificar e classificar as fraudes, reduzindo posteriormente o número de fraudes que acabam ocorrendo e que não são identificadas a tempo. Para resolver esse tipo de problema, vamos trabalhar com o que conhecemos como classes desbalanceadas. Utilizaremos técnicas de oversampling e undersampling para mitiga esses problemas. 

Mais informações: https://imbalanced-learn.org/stable/auto_examples/index.html#general-examples

<h2> Dados</h2>
Os conjuntos de dados contêm transações realizadas com cartões de crédito em setembro de 2013 por titulares de cartões europeus. Esse conjunto de dados apresenta transações que ocorreram em 2 dias, nas quais temos 492 fraudes em 284.807 transações. O conjunto de dados é altamente desequilibrado, a classe positiva (fraude) representa 0,172% de todas as transações.

O cojunto de dados contém apenas variáveis numéricas que são o resultado de uma transformação de PCA. Infelizmente, devido a questões de confidencialidade , não são fornecidos os detalhes dos atributos e suas características.

Atributos V1, V2,… V28 são os principais componentes obtidos com o PCA, as únicas características que não foram transformadas com o PCA são 'Tempo' e 'Valor' (Valor da transação).

- O atributo 'Time' contém os segundos entre cada transação e a primeira transação no conjunto de dados.

- O atributo 'Amount' é o valor da transação.

- O atributo 'Class' é a variável de resposta e assume o valor 1 em caso de fraude e 0 em caso de não fraude.

<img src="creditcard.png">

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import seaborn as sns 
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import NearMiss
from tqdm import tqdm_notebook

from sklearn.metrics import classification_report, recall_score, precision_score ,average_precision_score, plot_precision_recall_curve
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix
from sklearn.impute import SimpleImputer
from yellowbrick.classifier import PrecisionRecallCurve, ConfusionMatrix
from sklearn.model_selection import train_test_split, cross_validate ,KFold, StratifiedShuffleSplit, StratifiedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler, PowerTransformer, QuantileTransformer, RobustScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier 
import shap
shap.initjs()

%matplotlib inline 
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

In [ ]:
# Importando os dados
data = pd.read_csv('creditcard.csv')
data.head()

Como podemos ver, todas as colunas estão já normalizadas, em escala e anonimizadas. Assim não conseguimos saber o que significa cada atributo.

In [ ]:
data.shape

Temos um total de 31 colunas e 284 mil registros.

In [ ]:
# Análise estatística básica
data.describe()

In [ ]:
# Verificando a distribuição das classes 0 e 1
sns.countplot(x=data['Class'], palette='Pastel2')
print('Normal: {} |  Fraud: {}'.format(data[data['Class']==0].shape[0] , data[data['Class']==1].shape[0]))

In [ ]:
# As classes são muito distorcidas, precisamos resolver esse problema. E isso é o principal objetivo desse notebook da Aula 5.
print('Não Fraudes:', round(data['Class'].value_counts()[0]/len(data) * 100,2), '% do dataset')
print('Fraudes:', round(data['Class'].value_counts()[1]/len(data) * 100,2), '% do dataset')

**Nota:**  Observe como nosso conjunto de dados original está desequilibrado! A maioria das transações não são fraudes. Se usarmos esse dataframe como base para nossos modelos e análises preditivas, podemos obter muitos erros e nossos algoritmos provavelmente se ajustarão, pois "suporão" que a maioria das transações não é fraude. Mas não queremos que nosso modelo assuma, queremos que nosso modelo detecte padrões que dão sinais de fraude!

Ao ver as distribuições, podemos ter uma ideia de quão distorcidos estão as variáveis, também podemos ver outras distribuições de outras variáveis. Existem técnicas que podem ajudar as distribuições a serem menos distorcidas e que serão implementadas neste notebook. Vamos fazer engenharia de atributos com esses dados, a fim de maximizar as métricas : <b> ROC AUC | Precision | Recall </b>.

Tentar medir o desempenho do modelo com Acurácia seria um grande erro, pois teríamos uma alta acurácia, o que de fato não resolveria nosso problema, pois o conjunto possui classes desbalanceadas. Assim, vamos focar nessas três métricas listadas acima, que não variam com o desequilíbrio das classes.

### Análise dos Dados

Primeiro iremos verificar a distribuição de segundos, tentando identificar o período de tempo em que uma transação fraudulenta é feita, em comparação com transações normais.

In [ ]:
# Vamos entender a coluna Time primeiro.
print("Tempo inicial em segundos:", data['Time'].min())
print("Tempo final (em segundos)", data['Time'].max())
print('Tempo em dias:', 172792/86400)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,4))

amount_val = data['Amount'].values
time_val = data['Time'].values

sns.distplot(amount_val, ax=ax[0], color='b')
ax[0].set_title('Distribuição do Valor da Transação', fontsize=14)
ax[0].set_xlim([min(amount_val), max(amount_val)])

sns.distplot(time_val, ax=ax[1], color='g')
ax[1].set_title('Distribuição do Tempo das Transações (em segundos)', fontsize=14)
ax[1].set_xlim([min(time_val), max(time_val)])

plt.show()

In [ ]:
plt.figure(figsize=(16,12))

plt.subplot(2,1,1)
plt.title('Temporariedade das transações não-fraudes')
sns.distplot(data[data['Class']==0]['Time'], color='green')

print('\n')
print('\n')

plt.subplot(2,1,2)
plt.title('Temporariedade das transações fraudes')
sns.distplot(data[data['Class']==1]['Time'], color='red')

Observando as duas distribuições, podemos ver que o período de uma transação rotulada como Normal, ocorre em média de forma trivial o que é considerado algo normal. Já as transações fraudulentas ocorrem com menor frequência (obviamente) e com uma lacuna bem importante nos tempos entre 100k a 130k. 

Precisamos verificar posteriormente, se uma transação fraudulenta ocorre mais de uma vez, com os mesmos dados da transação.

In [ ]:
# Existem transações repetidas?

# Existe um método duplicated() no pandas
# Link: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

fraude = data[data['Class']==1].loc[data.duplicated(keep=False)]
print('Transações repetidas: {} '.format(len(fraude)))
print('\n')
fraude

<br>
Foram <b> 32 transações repetidas </b> nestes dois dias, há casos em que ocorreram três fraudes com o mesmo valor. Algo que chama a atenção são os baixos valores das transações que houve um número maior de fraudes, talvez porque o fraudador considere o risco de ser pego em relação ao número de tentativas.
<br>

In [ ]:
# cmap
cmap = sns.diverging_palette(120, 40, sep=20, as_cmap=True, center='dark')

In [ ]:
# Correlação
corr = data.corr(method='pearson')

fig, ax = plt.subplots(figsize=(23,15))

# cmap=Greys

plt.title('Matriz de correlação', fontsize=16)
print('\n')
correlacao = sns.heatmap(corr, annot=True, cmap='Blues', ax=ax, lw=3.3, linecolor='lightgray')
correlacao

Observando as correlações de Pearson, podemos ver que não há grandes correlações positivas, apenas algumas características ultrapassam <b> 0,30 de correlação, que são <b> V7 com Amount e V20 com Amount </b>. Temos também correlações negativas interessantes, V2 com Amount (-0,53) e V5 com Amount (-0,39).

<br>

#### Distribuições

Vamos explorar um pouco as distribuições, das variáveis mascaradas <b> (V) </b>, para ver como elas se comportam, nesses dois dias de transações financeiras.

In [ ]:
# 
cols_names = data.drop(['Class', 'Amount', 'Time'], axis=1)
idx = 0

# Separando as classes 0 e 1
fraud = data[data['Class']==1]
normal = data[data['Class']==0]

# Plotando a figura  
fig, ax = plt.subplots(nrows=7, ncols=4, figsize=(18,18))
fig.subplots_adjust(hspace=1, wspace=1)

for col in cols_names:
    idx += 1
    plt.subplot(7, 4, idx)
    sns.kdeplot(fraud[col], label="Normal", color='green', shade=True)
    sns.kdeplot(normal[col], label="Fraud", color='red', shade=True)
    plt.title(col, fontsize=11)
    plt.tight_layout()

Todas as distribuições de variáveis que possuem uma "máscara", não sabemos a real representação dessas variáveis porque elas estão ocultas, mas através da distribuição com a densidade da informação, dá para ver muito bem as curvas de cada uma, comparando com uma normal transação ou fraude.


In [ ]:
# Qual o valor médio das transações fraudulentas
print('Média de valor de fraude: {} | Média de valor normal: {}'.format(fraud['Amount'].mean() , normal['Amount'].mean()))

In [ ]:
# Vamos validar essas informações
fraud['Amount'].describe()

In [ ]:
normal['Amount'].describe()

In [ ]:
# Qual o maior valor de fraude? e do valor normal?
print('O valor mais alto de fraude é: {}  | Maior valor de transação normal: {}'.format(fraud['Amount'].max(), normal['Amount'].max()))

In [ ]:
# Distribuição da quantidade de transações
plt.figure(figsize=(10,5))
plt.title('Transações', fontsize=14)
plt.grid(False)
sns.kdeplot(data['Amount'], color='lightblue', shade=True)

In [ ]:
# Vamos fazer a análise exploratória utilizando o Dataprep
from dataprep.eda import plot, create_report

# Utilizando o método plot para construção do relatório EDA de forma automática dos nomes dos restaurantes
plot(data)

### Análise dos dados

Após essa etapa de exploração dos dados, conseguimos ter bons insights sobre o todo, transações fraudulentas tendem a ter valores maiores que transações normais, fraudes repetidas contém valores menores.

Iremos criar vários experimentos, em busca de um modelo robusto que atinja nosso objetivo.

<br>
<hr>
<br>

## Modelo base (sem balanceamento) - Regressão Logística

Vamos criar um modelo base, para poder comparar os próximos resultados de outros experimentos, com esse modelo puro e simples que vamos construir.

Ou seja, nesse modelo não será aplicada nenhuma técnica de balanceamento.

In [ ]:
# Dividindo as classes em fraude e não-fraude

X = data.drop('Class', axis=1)
y = data['Class']

# Treino / Teste (70/30)
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

# StandardScaler 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Encoder 
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
# Modelo base

baseline = LogisticRegression(random_state=42)
baseline.fit(X_train, y_train)
y_baseline = baseline.predict(X_test)

# probabilidades
y_proba_baseline = baseline.predict_proba(X_test)[:,1]

print(classification_report(y_test, y_baseline))
print('\n')
print('AUC: {}%'.format(roc_auc_score(y_test, y_proba_baseline)))
print('Precision-Recall: {}'.format(average_precision_score(y_test, y_proba_baseline)))

### Resultados do modelo base

* AUC: 0.97
* AUPRC: 0.78
* Precision: 0.88
* Recall: 0.63 (valor extremamente baixo)

Os resultados nos dão uma ideia de quanto podemos melhorar o modelo de base, para um modelo melhor. Vamos tentar executar outros experimentos para obter um melhor desempenho.
Lembrando que o valor do AUC varia de 0,0 até 1,0 e o limiar entre a classe é 0,5. Ou seja, acima desse limite, o algoritmo classifica em uma classe e abaixo na outra classe.

Podemos ver como o recall teve um valor extremamente baixo. Isso significa que ele não acerta muito bem a classe 1 (fraude).

<hr>
<br>

### Dessa forma, vamos rodar 3 experimentos:

- **Experimento 1**: Random Forest + Oversampling (Smote)
- **Experimento 2**: XGBoost + Oversampling (Smote)
- **Experimento 3**: XGBoost + Undersampling (Nearmiss)

### Experimento 1: Random Forest + Oversampling (Smote)

Vamos aplicar técnicas de balanceamento de dados, e também validar nossos modelos a partir da validação cruzada. Vamos primeiro aplicar um Random Forest, combinado com uma técnica de OverSampling que basicamente irá, criar dados sintéticos na classe minoritária que é a classe de fraude, vamos testar essa abordagem e ver o desempenho do modelo. 



<p align=center>
<img src="https://blog.strands.com/hs-fs/hubfs/Screenshot%202019-07-18%20at%2014.15.15.png?width=600&name=Screenshot%202019-07-18%20at%2014.15.15.png" width="60%"></p>


<br>
<br>

Vamos então construir um modelo Random Forest com 200 árvores combinado com a técnica OverSampling <b> SMOTE </b> e ver os resultados das métricas que queremos otimizar, que são:

* AUC 
* Precision
* Recall 


In [ ]:
# Vamos fazer o balanceamento dentro da validaçõ cruzada

from imblearn.over_sampling import SMOTE
X = data.drop('Class', axis=1)
y = data['Class']

# Validação cruzada 
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Train: ',train_index.shape[0])
      print('Test: ', test_index[0])

      # Separando as classes
      X = data.drop('Class', axis=1)
      y = data['Class']

      # OverSampling SMOTE
      smote = SMOTE(random_state=42)
      X, y = smote.fit_resample(X, y)
      print('Normal: {}  |  Fraud: {}'.format(np.bincount(y)[0], np.bincount(y)[1])) #Contar o número de ocorrências de cada valor na matriz de inteiros não negativos.

      # Dividindo os dados  
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      # Pré-processamento
      scaler = QuantileTransformer(random_state=42)
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      # Construindo o modelo 
      forest = RandomForestClassifier(n_estimators=200, max_depth=13, min_samples_split=9,
                                    random_state=42)
      forest.fit(X_train, y_train)
      y_pred_forest = forest.predict(X_test)
      y_proba_forest = forest.predict_proba(X_test)[:,1]


      # Métricas 
      print('\n')
      print(classification_report(y_test, y_pred_forest))
      print('--------------'*5)
      print('\n')
      auc_forest = roc_auc_score(y_test, y_proba_forest)
      precision_forest = precision_score(y_test, y_pred_forest)
      recall_forest = recall_score(y_test, y_pred_forest)
      auprc_forest = average_precision_score(y_test, y_proba_forest)

In [ ]:
# Métricas do Experimento 1
print('Random Forest')
print('\n')

print('AUC: ', np.mean(auc_forest))
print('Precision: ', np.mean(precision_forest))
print('Recall: ', np.mean(recall_forest))
print('Precision-Recall: ', np.mean(auprc_forest))

print('\n')
print('\n')

# Curva ROC do Random Forest 
auc_forest = np.mean(auc_forest)
fpr_forest, tpr_forest, thresholds_forest = roc_curve(y_test, y_proba_forest)

# Plotando o gráfico 
plt.figure(figsize=(12,7))
plt.plot(fpr_forest, tpr_forest, color='blue', label='AUC: {}'.format(auc_forest))
plt.fill_between(fpr_forest, tpr_forest, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC Random Forest', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()


### Resultados do Experimento 1

AUC:  0.972695203329736  
Precision:  0.927710843373494  
Recall:  0.7857142857142857  
Precision-Recall:  0.8257579754337053  

Em resumo, o modelo obteve resultados satisfatórios. A AUC do modelo foi alta, na validação o resultado foi <b> 97% </b>, porém as demais métricas não atingiram um valor tão bom de melhoria quanto se esperava ao aplicar a técnica de balanceamento. A precisão foi alta, mas o Recall não aumentou muito em comparação com a modelo base. O Precision-Recall foi de 0.82 (contra 0.78 no modelo base) e é a principal métrica em nossa avaliação e é difícil de otimizar, porque queremos um modelo que possa separar bem a fraude de uma transação normal, e também para identificar bem uma fraude quando ela realmente ocorre.

<hr>

### Experimento 2: XGBoost + Oversampling (Smote)

XGBoost é um algoritmo de Aprendizado de Máquina, baseado em uma árvore de decisão e que usa uma estrutura de Gradient Boosting

Na previsão de problemas envolvendo dados não estruturados, como imagens, textos e vídeos, as redes neurais artificiais tendem a superar todos os outros algoritmos ou frameworks. No entanto, quando se trata de dados estruturados / tabulares, os algoritmos baseados em árvore de decisão são considerados os melhores em sua classe no momento.


Vamos agora aplicar a técnica OverSampling com o XGboost e para isso vamos usar a técnica de <b>SMOTE</b> que será responsável pela aplicação do OverSampling, igualando as classes maximizando as proporções da classe 1.

<br>

In [ ]:
from imblearn.over_sampling import SMOTE
X = data.drop('Class', axis=1)
y = data['Class']

# Validação
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
 
precision_xgboost = []
recall_xgboost = []
auc_xgboost = []
precision_recall_xgboost = []

fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Train: ',train_index.shape[0])
      print('Test: ', test_index[0])
    
      # OverSampling com SMOTE 
      smt = SMOTE(random_state=42)
      X, y = smt.fit_resample(X, y)
      print('Normal: {}  |  Fraud: {}'.format(np.bincount(y)[0], np.bincount(y)[1]))
     

      # Dividindo o dataset 
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      
      # Pré-processamento
      scaler = QuantileTransformer()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)

      # XGboost 
      xgb = XGBClassifier(n_estimators=300, max_delta_step=1 ,eval_metric='aucpr', 
                          cpu_history='gpu', random_state=42)
      xgb.fit(X_train, y_train)
      y_pred = xgb.predict(X_test)
  

      # Métricas 
      precision_recall_xgboost = average_precision_score(y_test, y_pred)
      precision_xgboost = precision_score(y_test, y_pred)
      recall_xgboost = recall_score(y_test, y_pred)
      auc_xgboost  = roc_auc_score(y_test, y_pred)
      print('Precision-Recall: ', average_precision_score(y_test, y_pred))
      print('\n')
      print('\n')



# Validação Final
print('Precision-Recall: ', np.mean(precision_recall_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('AUC: ', np.mean(auc_xgboost))

In [ ]:
# Validação do XGboost + SMOTE
print('XGboost')
print('\n')

print('AUC: ', np.mean(auc_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision-Recall: ', np.mean(precision_recall_xgboost))


print('\n')
print('\n')

# Curva ROC para XGBoost
roc_auc_xgboost = np.mean(auc_xgboost)
fpr_xgboost, tpr_xgboost, thresholds_xgboost = roc_curve(y_test, y_pred)

# Plotando a figura 
plt.figure(figsize=(12,7))
plt.plot(fpr_xgboost, tpr_xgboost, color='blue', label='AUC: {}'.format(roc_auc_xgboost))
plt.fill_between(fpr_xgboost, tpr_xgboost, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC XGboost', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()

### Resultados do Experimento 2

AUC:  0.8978712530331584  
Precision:  0.8863636363636364  
Recall:  0.7959183673469388  
Precision-Recall:  0.705824215761466  

Em resumo, o modelo 2 com XGBoost obteve resultados satisfatórios. A AUC do modelo foi alta, na validação o resultado foi <b> 89% </b>, porém as demais métricas não atingiram um valor tão bom de melhoria quanto se esperava ao aplicar a técnica de balanceamento. A precisão foi alta, mas o Recall não aumentou muito em comparação com a modelo base. O Precision-Recall foi de 0.70 (contra 0.78 no modelo base).

<hr>
<br>

### Experimento 3: XGBoost + Undersampling (Nearmiss)


Combinaremos o XGboost com a técnica Undersampling NearMiss. Usaremos os mesmos parâmetros do modelo anterior, neste caso minimizaremos a classe majoritária para uma proporção igual à nossa classe minoritária.

In [ ]:
X = data.drop('Class', axis=1)
y = data['Class']

# UnderSampling NearMiss
under = NearMiss()
X,y = under.fit_resample(X, y)

# Validacão
KFold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

resultados = []
fold = 0
for train_index, test_index in KFold.split(X,y):
      fold += 1 
      print('Fold: ', fold)
      print('Train: ',train_index.shape[0])
      print('Test: ', test_index[0])

      # Classe desbalanceada
      print('Normal: {} | Fraud: {}'.format(np.bincount(y)[0], np.bincount(y)[1]))


      # Dividindo o dataset
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y[train_index], y[test_index] 

      
      # Pré-processamento
      scaler = StandardScaler()
      X_train = scaler.fit_transform(X_train)
      X_test = scaler.transform(X_test)


      # Encoder 
      encoder = LabelEncoder()
      y_train = encoder.fit_transform(y_train)
      y_test = encoder.transform(y_test)


      # XGboost 
      xgb = XGBClassifier(n_estimators=300, max_delta_step=1 ,eval_metric='aucpr', 
                          cpu_history='gpu', random_state=42)
      xgb.fit(X_train, y_train)
      y_pred = xgb.predict(X_test)

    # Métricas de avaliação
      precision_recall_xgboost = average_precision_score(y_test, y_pred)
      precision_xgboost = precision_score(y_test, y_pred)
      recall_xgboost = recall_score(y_test, y_pred)
      auc_xgboost  = roc_auc_score(y_test, y_pred)
      print('Precision-Recall: ', average_precision_score(y_test, y_pred))
      print('\n')
      print('\n')



# Validação Final  
print('Precision-Recall: ', np.mean(precision_recall_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('AUC: ', np.mean(auc_xgboost))

In [ ]:
# Métricas do Experimento 3: XGboost + NearMiss  
print('XGboost')
print('\n')

print('AUC: ', np.mean(auc_xgboost))
print('Precision: ', np.mean(precision_xgboost))
print('Recall: ', np.mean(recall_xgboost))
print('Precision-Recall: ', np.mean(precision_recall_xgboost))


print('\n')
print('\n')

# Curva ROC random forest 
roc_auc_xgboost = np.mean(auc_xgboost)
fpr_xgboost, tpr_xgboost, thresholds_xgboost = roc_curve(y_test, y_pred)

# Plotando a figura 
plt.figure(figsize=(12,7))
plt.plot(fpr_xgboost, tpr_xgboost, color='blue', label='AUC: {}'.format(roc_auc_xgboost))
plt.fill_between(fpr_xgboost, tpr_xgboost, color='skyblue', alpha=0.3)
plt.plot([0,1], [0,1], color='black', ls='--', label='Reference line')
plt.xlabel('False Positive Rate', fontsize=14)
plt.ylabel('True Positive Rate', fontsize=14)
plt.title('ROC XGboost', fontsize=16)
plt.legend(loc=4, fontsize=14)
plt.grid(False)
plt.show()

### Resultados do Experimento 3

AUC:  0.933673469387755  
Precision:  0.956989247311828  
Recall:  0.9081632653061225  
Precision-Recall:  0.9150208470484968  

A melhor combinação até agora foi o modelo 3 onde conseguimos obter valores muito bons para as principais métricas analizadas.

In [ ]:
# Matriz de confusão

plt.figure(figsize=(15,10))

plt.subplot(2, 1, 1)
plt.title('Matriz de confusão XGboost', fontsize=15)
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, cmap='Pastel2')

print('\n')
print('\n')

In [ ]:
import shap

explainer = shap.Explainer(xgb)
shap_values = explainer(X)
shap.summary_plot(shap_values,X, max_display=30)



Conseguimos visualizar nossa matriz de confusão, que mostra que o modelo conseguiu classificar bem as fraudes, minimizamos o Falso Negativo que contém Fraudes que foram classificadas incorretamente.

Visualizando as variáveis mais importantes, que o modelo XGboost + NearMiss identificou, <b> V14 </b> é considerada pelo modelo como o atributo mais importante, é a de maior magnitude. Infelizmente, por estarem mascaradas, não conseguimos identificar do que se trata.


<hr>

### Conclusão

Lidar com projetos onde as classes são desbalanceadas é sempre um desafio, e lidar com um problema tão delicado que é a fraude bancária, o maior desafio foi refletir sobre os insights que vimos sobre os dados, entender quais técnicas e algoritmos aplicar era de suma importância. É muito sensível lidar com transações que ocorrem em uma frequência tão alta e rápida. Dessa forma, resultado final foi satisfatório, onde conseguimos criar um modelo com um <b> Precision-Recall de: 98% </b> usando o <b> Combinação XGboost + NearMiss </b>, identificando uma fraude entre as transações, e que também conseguisse separar uma transação normal de uma fraude. 

<hr>
